In [12]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

display(HTML("<style>.container { width:95% !important; }</style>"))

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from tqdm.auto import tqdm

sns.set_style("ticks")

# Japanese font installation
# 1) `$ mkdir $HOME/.fonts`
# 2) Download Osaka.ttf tp $HOME/.fonts/
# 3) `$ fc-cache`
# 4) (optional) Clear matplotlib cache dir, find the dir with `matplotlib.get_cachedir()`
plt.rcParams['font.family'] = 'Osaka'

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

np.set_printoptions(suppress=True)

# Train

In [ ]:
import sys

sys.path.append("../")

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from libs.dataset import RiidDataset, get_data_loader
from libs.model import Predictor

import pickle

dataset = RiidDataset(data_root_dir="../dataset/processed", split="train")

dataloader = get_data_loader(dataset=dataset, batch_size=16,
                             shuffle=False, num_workers=8)

bundle_id_idx = pickle.load(open("../dataset/processed/indexes/bundle_id_idx.pickle", "rb"))
part_idx = pickle.load(open("../dataset/processed/indexes/part_idx.pickle", "rb"))
content_id_idx = pickle.load(open("../dataset/processed/indexes/content_id_idx.pickle", "rb"))
type_idx = pickle.load(open("../dataset/processed/indexes/type_idx.pickle", "rb"))


config = dict(content_id_size=len(content_id_idx)+1,
              content_id_dim=256,
              bundle_id_size=len(bundle_id_idx)+1,
              bundle_id_dim=128,
              feature_dim=204,
              lstm_hidden_dim=256,
              lstm_num_layers=2,
              lstm_dropout=0.3,
              emb_dropout=0.3,
              output_dropout=0.3,
              lr=0.05)
# config = dict(content_id_size=len(content_id_idx)+1,
#               content_id_dim=8,
#               bundle_id_size=len(bundle_id_idx)+1,
#               bundle_id_dim=8,
#               feature_dim=204,
#               lstm_hidden_dim=8,
#               lstm_num_layers=1,
#               lstm_dropout=0.3,
#               emb_dropout=0.3,
#               output_dropout=0.3,
#               lr=0.001)

model = Predictor(**config)
print(model)
total_params = sum(p.numel() for p in model.parameters())
print(f"total param: {total_params:,}")

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"total learnable param: {total_params:,}")

save_top_k = 5
max_epochs = 1
train_loader = dataloader

checkpoint_callback = ModelCheckpoint(
    monitor='train_loss',
    dirpath='./logs',
    save_top_k=save_top_k,
    mode='min')

logger = TensorBoardLogger(save_dir="./tensorboard_logs", name='riid')

trainer = pl.Trainer(max_epochs=max_epochs,
                     callbacks=[checkpoint_callback],
                     logger=logger)
trainer.fit(model, train_loader)

/Users/phiradet/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Checkpoint directory ./logs exists and is not empty. With save_top_k=5, all files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
/Users/phiradet/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)

  | Name           | Type      | Params
---------------------------------------------
0 | content_id_emb | Embedding | 3 M   
1 | bundle_id_emb  | Embedding | 1 M   
2 | lstm           | LSTM      | 1 M   
3 | hidden2logit   | Linear    | 257   


Predictor(
  (content_id_emb): Embedding(13942, 256, padding_idx=0)
  (bundle_id_emb): Embedding(9767, 128, padding_idx=0)
  (lstm): LSTM(588, 256, num_layers=2, batch_first=True, dropout=0.3)
  (hidden2logit): Linear(in_features=256, out_features=1, bias=True)
)
total param: 6,212,225
total param: 6,212,225
